# BAR Vignette

In [ ]:
# Bayesian Autoregressive (BAR) Model Demonstration

# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
from bar_module import BayesianAR

# Step 1: Generate synthetic time series data
np.random.seed(42)
n_points = 200
time = np.linspace(0, 4 * np.pi, n_points)
series = np.sin(time) + np.random.normal(scale=0.3, size=n_points)

plt.figure(figsize=(10, 4))
plt.plot(series, label="Synthetic Time Series")
plt.title("Generated Synthetic Time Series Data")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid()
plt.show()

# Step 2: Initialize the BayesianAR model
model = BayesianAR(p=3)

# Step 3: Fit the model to the synthetic data
model.fit(series)

# Step 4: Forecast future points
n_forecast = 20
forecasted_values = model.forecast(series[-3:], steps=n_forecast)

# Step 5: Visualize results
plt.figure(figsize=(10, 4))
plt.plot(series, label="Observed Data", marker='o')
forecast_range = np.arange(n_points, n_points + n_forecast)
plt.plot(forecast_range, forecasted_values, label="Forecasted Data", marker='x', linestyle='--')
plt.xlabel("Time")
plt.ylabel("Value")
plt.title("Bayesian AR Forecast")
plt.legend()
plt.grid()
plt.show()